In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm

In [2]:
df = pd.read_csv('/kaggle/input/super-ai-engineer-5-question-and-answer/QA/train.csv')
df.head()

,question,A,B,C,D,correct
0,I can't believe that once the answer was revealed,we all fell through the floor.,we still didn't understand the question.,no one had revealed to us the answer.,we wished to go pet that dog.,B
1,"But Emma was the Emperor’s Hand, recruited and...",water,instincts,food,hands,B
2,James just won the lottery. He,will go and tell the ghosts in his house,will go home to eat dinner,is angry he won the lottery,feels happy,D
3,Janice fights with her sister. Janice,throws a mountain at her sister,drinks nitro,punches her,sisters her fight attack,C
4,A dog is running around in the park. The dog,is chasing after a ghost,is running after a ball,is running to get a lot of exercise it doesn't...,is searching for food,B


## Unsloth setup
- transformer-based: doesn't perform well

In [3]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 1.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 71.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 6.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.9/126.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 MB 42.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.2 MB/s eta 0:00:

In [4]:
from unsloth import FastLanguageModel
from datasets import Dataset
import torch

max_seq_length = 256
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-7B",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    dtype=None,
    full_finetuning=False,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.18: Fast Qwen2 patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/106k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.18 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [8]:
def convert_row_to_sft(row):
    instruction = "Choose the best answer to complete the sentence."

    input_text = f"""Question: {row['question']}
A) {row['A']}
B) {row['B']}
C) {row['C']}
D) {row['D']}"""

    output_text = row['correct']  # Just the letter: A/B/C/D

    return {
        "instruction": instruction,
        "input": input_text,
        "output": output_text
    }


sft_data = [convert_row_to_sft(row) for _, row in df.iterrows()]
dataset = Dataset.from_list(sft_data)
print(dataset[0])

{'instruction': 'Choose the best answer to complete the sentence.', 'input': "Question: I can't believe that once the answer was revealed\nA) we all fell through the floor.\nB) we still didn't understand the question.\nC) no one had revealed to us the answer.\nD) we wished to go pet that dog.", 'output': 'B'}


In [9]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request.
### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, prompt, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, prompt, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset[0]

Map:   0%|          | 0/1857 [00:00<?, ? examples/s]

{'instruction': 'Choose the best answer to complete the sentence.',
 'input': "Question: I can't believe that once the answer was revealed\nA) we all fell through the floor.\nB) we still didn't understand the question.\nC) no one had revealed to us the answer.\nD) we wished to go pet that dog.",
 'output': 'B',
 'text': "Below is an instruction that describes a task, paired with an input that provides further context. \nWrite a response that appropriately completes the request.\n### Instruction:\nChoose the best answer to complete the sentence.\n\n### Input:\nQuestion: I can't believe that once the answer was revealed\nA) we all fell through the floor.\nB) we still didn't understand the question.\nC) no one had revealed to us the answer.\nD) we wished to go pet that dog.\n\n### Response:\nB<|endoftext|>"}

## Model Training

In [11]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 30,
        learning_rate = 5e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/1857 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,857 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 40,370,176/7,000,000,000 (0.58% trained)


Step,Training Loss
1,1.520800
2,1.573900
3,1.559300
4,1.340500
5,1.332200
6,1.183600
7,1.123900
8,1.153100
9,1.279100
10,1.034200


KeyboardInterrupt: 

## Predictions

In [12]:
test_df = pd.read_csv('/kaggle/input/super-ai-engineer-5-question-and-answer/QA/test.csv', 
                      index_col='id')
print(test_df.shape)
test_df.head()

(919, 5)


,question,A,B,C,D
id,,,,,
0,Pablo calls running slurping. If Pablo is not ...,Jogging,Sprinting,doing cardio.,Sitting
1,"Bob is vegetarian but his wife eats meat. So, ...",chicken,fish,steak,vegetables
2,There is a fly in the house and Joseph wants t...,flies to Korea without any flies.,tells the fly to kill himself.,kills a wizard so that he can fly.,sprays insect spray in the house.
3,Today I was asked whether I would kill one man...,"I would not kill the man, because he has been ...","I would kill the man, because poverty is a big...","I would not kill the man, because the ends do ...","I would kill the man, because when life gives ..."
4,today is saturday and yesterday was,friday,monday,sunday,everyday


In [13]:
def sft_test(row):
    instruction = "Choose the best answer to complete the sentence."

    input_text = f"""Question: {row['question']}
A) {row['A']}
B) {row['B']}
C) {row['C']}
D) {row['D']}"""


    return {
        "instruction": instruction,
        "input": input_text,
    }


sft_data = [sft_test(row) for _, row in test_df.iterrows()]
test_set = Dataset.from_list(sft_data)
print(test_set[0])

{'instruction': 'Choose the best answer to complete the sentence.', 'input': 'Question: Pablo calls running slurping. If Pablo is not slurping he could be\nA) Jogging\nB) Sprinting\nC) doing cardio.\nD) Sitting'}


In [14]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

def extract_response(output_str: str) -> str:
    if "### Response:" in output_str:
        response = output_str.split("### Response:")[-1].strip()
    else:
        response = output_str.strip()

    # Remove any special tokens (like <|endoftext|>)
    response = response.replace("<|endoftext|>", "").strip()

    # If it's a multiple choice letter, just keep the first letter
    if response and response[0] in ["A", "B", "C", "D"]:
        return response[0]
    return response


def generate_answer(dataset):
    answer = []
    for prompt in tqdm(dataset):
        inputs = tokenizer(
        [
            alpaca_prompt.format(
                prompt['instruction'],
                prompt['input'],
                "", # output - leave this blank for generation!
            )
        ], return_tensors = "pt").to("cuda")
        
        outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
        output = tokenizer.batch_decode(outputs)
        answer.append(extract_response(output[0]))

    return answer

ans = generate_answer(test_set)
ans[:5]

100%|██████████| 919/919 [05:33<00:00,  2.75it/s]


['A', 'D', 'D', 'C', 'A']

In [15]:
submission = pd.read_csv('/kaggle/input/super-ai-engineer-5-question-and-answer/QA/sample_submission.csv')
submission.loc[2:, 'answer'] = ans[2:]
submission.head()

,id,answer
0,0,D
1,1,D
2,2,D
3,3,C
4,4,A


In [16]:
submission.to_csv('qwen_7B_2.csv', index=False)